In [1]:
import os,io,requests,json,zipfile,pandas as pd

# Setting user & survey parameters
Assumes user has api token generated and stored in text file at 'U:\keys\qualtrics-api-token.txt'

List as many or as few qualtrics survey IDs as needed in surveyIdList

Currently requires re-executing the script for each survey, as the zipfile extract fails out once per survey. I do not understand the zipfile functionality enough to troubleshoot at the moment.

In [3]:
apiToken = open(os.path.join('u:/','keys','qualtrics-api-token.txt'),'r').read()
surveyIdList = ["SV_bsl9vF012E6Thbv","SV_bBNxUVhoGFcyKot","SV_2gAX1ty1UjOeKBT","SV_d4o0ZLBDB8GxpwV"]
fileFormat = "csv"
dataCenter = 'ca1'
output_dir = os.path.join('..','sourcedata','qualtrics')

# Setting static parameters

In [4]:
requestCheckProgress = 0
progressStatus = "in progress"
baseUrl = "https://{0}.qualtrics.com/API/v3/responseexports/".format(dataCenter)
headers = {
    "content-type": "application/json",
    "x-api-token": apiToken,
    }

# Executing Request
If executing in Jupyter, this is the cell that will throw an error for each survey.

Simply re-execute until it stops throwing errors...

In [8]:
for surveyId in surveyIdList:
    # Step 1: Creating Data Export
    downloadRequestUrl = baseUrl
    downloadRequestPayload = '{"format":"' + fileFormat + '","surveyId":"' + surveyId + '"}'
    downloadRequestResponse = requests.request("POST", downloadRequestUrl, data=downloadRequestPayload, headers=headers)
    progressId = downloadRequestResponse.json()["result"]["id"]
    print(downloadRequestResponse.text)
    
    # Step 2: Checking on Data Export Progress and waiting until export is ready
    while requestCheckProgress < 100 and progressStatus is not "complete":
        requestCheckUrl = baseUrl + progressId
        requestCheckResponse = requests.request("GET", requestCheckUrl, headers=headers)
        requestCheckProgress = requestCheckResponse.json()["result"]["percentComplete"]
        print("Download is " + str(requestCheckProgress) + "% complete")
    
    # Step 3: Downloading file
    requestDownloadUrl = baseUrl + progressId + '/file'
    requestDownload = requests.request("GET", requestDownloadUrl, headers=headers, stream=True)
    
    # Step 4: Unzipping the file
    zipfile.ZipFile(io.BytesIO(requestDownload.content)).extractall(output_dir)
    print(surveyId,'done!')
print('All done!')

{"result":{"id":"ES_hriprdqtbb6ccci4tf2lg6ebuo"},"meta":{"httpStatus":"200 - OK","requestId":"e8e0927e-eac2-4958-b74f-aeed12bbee74"}}
SV_bsl9vF012E6Thbv done!
{"result":{"id":"ES_8i6uc703cjfho09cft4cqvv21g"},"meta":{"httpStatus":"200 - OK","requestId":"8687b379-692c-43bd-87f8-a5a5d7fb87da"}}
SV_bBNxUVhoGFcyKot done!
{"result":{"id":"ES_utis3nt50skrm3q0plmtv28h3h"},"meta":{"httpStatus":"200 - OK","requestId":"5a839eb5-cd9a-4e95-a3d8-f8226efc3958"}}
SV_2gAX1ty1UjOeKBT done!
{"result":{"id":"ES_4s44j7p9skddmnlefs1bv9dbmi"},"meta":{"httpStatus":"200 - OK","requestId":"4b9b5d8d-122a-4d8b-a98a-4334231a62f2"}}
SV_d4o0ZLBDB8GxpwV done!
All done!


# Read outputs, clean and re-write

In [9]:
columns_to_drop=[
    'StartDate','EndDate','Status','IPAddress','Finished',
    'RecipientLastName','RecipientFirstName','RecipientEmail',
    'LocationLatitude','LocationLongitude','LocationAccuracy',
    'ResponseID','ResponseSet','ExternalDataReference',
    'Score-weightedAvg','Score-weightedStdDev'
]

In [10]:
for f in os.listdir(output_dir):
    fpath = os.path.join(output_dir,f)
    if os.path.isfile(fpath):
        df = pd.read_csv(fpath)
        df = df[df['Finished'] == '1']
        df = df[[
            c for c in df.columns
            if not c.startswith('DO-')
            and not c.startswith('RO-')
            and not c in columns_to_drop
        ]]
        df.to_csv(fpath,index=False)